# 🎯 Emotion Detection Model Training (Custom CNN)
This notebook trains your own CNN model on a labeled emotion dataset stored in `dataset/train` and `dataset/test`. It saves the trained model and includes a preview + prediction block.

In [1]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:

# === CONFIG ===
image_size = (48, 48)
batch_size = 32
epochs = 10
train_dir = "dataset/train"
test_dir = "dataset/test"


In [3]:

# Load images using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical"
)

num_classes = len(train_generator.class_indices)
class_labels = list(train_generator.class_indices.keys())


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:

# === BUILD CNN MODEL ===
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 4, 4, 128)         0

In [5]:

# === TRAIN ===
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)


Epoch 1/10
898/898 [==============================] - 47s 50ms/step - loss: 1.7462 - accuracy: 0.2897 - val_loss: 1.5913 - val_accuracy: 0.3564
Epoch 2/10
898/898 [==============================] - 45s 50ms/step - loss: 1.5239 - accuracy: 0.4121 - val_loss: 1.3849 - val_accuracy: 0.4752
Epoch 3/10
898/898 [==============================] - 44s 48ms/step - loss: 1.3912 - accuracy: 0.4656 - val_loss: 1.3171 - val_accuracy: 0.4953
Epoch 4/10
898/898 [==============================] - 65s 72ms/step - loss: 1.3150 - accuracy: 0.4973 - val_loss: 1.2753 - val_accuracy: 0.5043
Epoch 5/10
898/898 [==============================] - 88s 97ms/step - loss: 1.2563 - accuracy: 0.5233 - val_loss: 1.2186 - val_accuracy: 0.5297
Epoch 6/10
898/898 [==============================] - 88s 98ms/step - loss: 1.2060 - accuracy: 0.5431 - val_loss: 1.2231 - val_accuracy: 0.5198
Epoch 7/10
898/898 [==============================] - 90s 100ms/step - loss: 1.1657 - accuracy: 0.5575 - val_loss: 1.1811 - val_accuracy

In [ ]:

# === SAVE MODEL ===
os.makedirs("models", exist_ok=True)
model_json = model.to_json()
with open("models/custom_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("models/custom_model_weights.h5")
print("✅ Model saved as JSON + weights")


In [ ]:

# === PLOTS ===
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()


In [ ]:

# === EVALUATION ===
test_generator.reset()
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))


In [ ]:

# === PREDICT ON NEW IMAGES (OPTIONAL) ===
image_folder = "images"
from keras.preprocessing.image import img_to_array

model = model_from_json(open("models/custom_model.json").read())
model.load_weights("models/custom_model_weights.h5")
print("✅ Custom model loaded.")

for img_file in os.listdir(image_folder)[:5]:
    path = os.path.join(image_folder, img_file)
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, image_size)
    face_img = resized.astype("float32") / 255.0
    face_img = img_to_array(face_img)
    face_img = np.expand_dims(face_img, axis=0)

    preds = model.predict(face_img)[0]
    label = class_labels[preds.argmax()]
    confidence = np.max(preds)

    print(f"{img_file} → {label} ({confidence*100:.2f}%)")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"{label} ({confidence*100:.2f}%)")
    plt.axis('off')
    plt.show()
